In [4]:
import pandas as pd
import boto3
import json
import psycopg2

from botocore.exceptions import ClientError
import configparser

def create_iam_role(iam, DWH_IAM_ROLE_NAME):
    '''
    IAM Role makes Redshift able to access S3 bucket (ReadOnly)
    '''
    
    try:
        print("1.1 Creating a new IAM Role") 
        dwhRole = iam.create_role(
            Path='/',
            RoleName=DWH_IAM_ROLE_NAME,
            Description = "Allows Redshift clusters to call AWS services on your behalf.",
            AssumeRolePolicyDocument=json.dumps(
                {'Statement': [{'Action': 'sts:AssumeRole',
                'Effect': 'Allow',
                'Principal': {'Service': 'redshift.amazonaws.com'}}],
                'Version': '2012-10-17'})
        )
    
    except Exception as e:
        print(e)
        
    print("1.2 Attaching Policy")
    
    iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                           PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                          )['ResponseMetadata']['HTTPStatusCode']
    
    print("1.3 Get the IAM role ARN")
    roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
    
    print(roleArn)
    return roleArn


def create_cluster(redshift, roleArn, DWH_CLUSTER_TYPE, DWH_NODE_TYPE, DWH_NUM_NODES, DWH_DB, DWH_CLUSTER_IDENTIFIER, DWH_DB_USER, DWH_DB_PASSWORD):
    
    '''
    CREATES A NEW REDSHIFT CLUSTER
    '''
    
    try:
        response = redshift.create_cluster(
        #HW
        ClusterType = DWH_CLUSTER_TYPE, #The type of the cluster; can either be multi or single-node
        NodeType = DWH_NODE_TYPE, #The node type to be provisioned for the cluster. e.g. dc2.large
        NumberOfNodes = int(DWH_NUM_NODES), #The number of compute nodes in the cluster.
        
        #Identifiers & Credentials
        DBName = DWH_DB, #The name of the first database to be created when the cluster is created.
        ClusterIdentifier = DWH_CLUSTER_IDENTIFIER, #A unique identifier for the cluster.
        MasterUsername = DWH_DB_USER, #The user name associated with the master user account for the cluster that is being created.
        MasterUserPassword = DWH_DB_PASSWORD, #The password associated with the master user account for the cluster that is being created.
        
        #ROLES ( for S3 access)
        IamRoles = [roleArn] #A list of AWS Identity and Access Management (IAM) roles that can be used by the cluster to access other AWS services.
        
        )
    
    except Exception as e:
        print(e)
        
def obtain_cluster_props(redshift, DWH_CLUSTER_IDENTIFIER):
    
    '''
    Get redshift cluster properties
    '''
    
    def prettyRedshiftProps(props):
        
        '''
        Obtain the redshift cluster's current status
        '''
        
        pd.set_option('display.max_colwidth', -1)
        KeysToShow = ['ClusterIdentifier', 'NodeType', 'ClusterStatus', 'MasterUsername', 'DBName', 'Endpoint', 'NumberOfNodes', 'VpcId']
        x = [(k,v) for k,v in props.items() if k in KeysToShow]
        print(pd.DataFrame(data=x, columns=['Key', 'Value']))
        
        return pd.DataFrame(data=x, columns=['Key', 'Value'])
    
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
    prettyRedshiftProps(myClusterProps)
    
    DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
    DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
    print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
    print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)
    
    return myClusterProps, DWH_ENDPOINT, DWH_ROLE_ARN

def open_tcp_port(ec2, myClusterProps, DWH_PORT):
    
    '''
    Open redshift tcp port to gain access to the cluster endpoint
    '''
    
    try:
        
        vpc = ec2.Vpc(id = myClusterProps['VpcId'])
        defaultSg = list(vpc.security_groups.all())[0]
        print(defaultSg)
        
        defaultSg.authorize_ingress(
            GroupName=defaultSg.group_name,
            CidrIp='0.0.0.0/0',
            IpProtocol='TCP',
            FromPort=int(DWH_PORT),
            ToPort=int(DWH_PORT)
        )
        
    except Exception as e:
        print(e)
        
def main():
    
    '''
    With the help of main(), we execute a ton of functionalities and also control the execution flow
    '''
    
    import configparser
    config = configparser.ConfigParser()
    config.read_file(open('dwh.cfg'))
    
    KEY                    = config.get('AWS','KEY')
    SECRET                 = config.get('AWS','SECRET')
    DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
    DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
    DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")
    DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
    DWH_DB                 = config.get("DWH","DWH_DB")
    DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
    DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
    DWH_PORT               = config.get("DWH","DWH_PORT")
    DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")
    
    (DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)
    
    df = pd.DataFrame({"Param":["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
                       "Value":[DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]})
    
    print(df)
    
    ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)
    
    s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )
    
    iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                      )
    
    redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)
    
    
    roleArn = create_iam_role(iam, DWH_IAM_ROLE_NAME)

    create_cluster(redshift, roleArn, DWH_CLUSTER_TYPE, DWH_NODE_TYPE, DWH_NUM_NODES, DWH_DB, DWH_CLUSTER_IDENTIFIER, DWH_DB_USER, DWH_DB_PASSWORD)

    myClusterProps = obtain_cluster_props(redshift, DWH_CLUSTER_IDENTIFIER)

    open_tcp_port(ec2, myClusterProps, DWH_PORT)
    

    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()

    print('Connected')

    conn.close()


if __name__ == "__main__":
    main()
    

                    Param       Value
0  DWH_CLUSTER_TYPE        multi-node
1  DWH_NUM_NODES           4         
2  DWH_NODE_TYPE           dc2.large 
3  DWH_CLUSTER_IDENTIFIER  dwhCluster
4  DWH_DB                  dwh       
5  DWH_DB_USER             dwhuser   
6  DWH_DB_PASSWORD         Passw0rd  
7  DWH_PORT                5439      
8  DWH_IAM_ROLE_NAME       dwhRole   
1.1 Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dwhRole already exists.
1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::698413254357:role/dwhRole
An error occurred (ClusterAlreadyExists) when calling the CreateCluster operation: Cluster already exists
                 Key  \
0  ClusterIdentifier   
1  NodeType            
2  ClusterStatus       
3  MasterUsername      
4  DBName              
5  Endpoint            
6  VpcId               
7  NumberOfNodes       

                                                                  

## After successfully running our redshift clusters, creating tables and loading data into them, LETS NOW Clean up our resources

In [14]:
def main():
    
    '''
    With the help of main(), we execute a ton of functionalities and also control the execution flow
    '''
    
    import configparser
    config = configparser.ConfigParser()
    config.read_file(open('dwh.cfg'))
    
    KEY                    = config.get('AWS','KEY')
    SECRET                 = config.get('AWS','SECRET')
    DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
    DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
    DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")
    DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
    DWH_DB                 = config.get("DWH","DWH_DB")
    DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
    DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
    DWH_PORT               = config.get("DWH","DWH_PORT")
    DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")
    
    (DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)
    
    ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)
    
    s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )
    
    iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                      )
    
    redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)
    
    #uncomment this when you want run this function for the first time
    #redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
    
    def prettyRedshiftProps(props):
        
        '''
        Obtain the redshift cluster's current status
        '''
        
        pd.set_option('display.max_colwidth', -1)
        KeysToShow = ['ClusterIdentifier', 'NodeType', 'ClusterStatus', 'MasterUsername', 'DBName', 'Endpoint', 'NumberOfNodes', 'VpcId']
        x = [(k,v) for k,v in props.items() if k in KeysToShow]
        df = pd.DataFrame(data=x, columns=['Key', 'Value'])
        return print(df)
    
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
    prettyRedshiftProps(myClusterProps)
    
    #uncomment this when you want run this function for the first time
    #iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
    #iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
    
if __name__ == "__main__":
    main()

ClusterNotFoundFault: An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster dwhcluster not found.

### The error above indicates that we have successfully deleted the Redshift Data Ware house cluster